In [1]:
import keras
import numpy as np
import scipy
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv3D, AveragePooling2D, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.utils import np_utils
#from sklearn.cross_validation import StratifiedKFold

Using TensorFlow backend.


In [2]:
PATH = os.getcwd()
print (PATH)

C:\Users\Smitha\Downloads\Classification-of-Hyperspectral-Image-master


In [3]:
# Load the Global values (windowSize, numPCAcomponents, testRatio) from the text file global_variables.txt
myFile = open('global_variables.txt', 'r') 
file = myFile.readlines()[:]


for line in file:

    if line[0:3] == "win":

        ds = line.find('=')
        windowSize = int(line[ds+1:-1],10)

    elif line[0:3] == "num":

        ds = line.find('=')
        numPCAcomponents = int(line[ds+2:-1],10)

    else:

        ds = line.find('=')
        testRatio = float(line[ds+1:])


In [28]:
# Global Variables
#windowSize = 5
#numPCAcomponents = 30
#testRatio = 0.25

# Load Training Dataset

In [4]:
X_train = np.load("X_trainPatches_" + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio)  + ".npy")

y_train = np.load("y_trainPatches_" + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio) + ".npy")

In [5]:
# Reshape into (numberofsamples, channels, height, width)
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[3], X_train.shape[1], X_train.shape[2]))

In [6]:
# convert class labels to on-hot encoding
y_train = np_utils.to_categorical(y_train)

In [7]:
# Define the input shape 
input_shape= X_train[0].shape
print(input_shape)

(30, 5, 5)


In [8]:
# number of filters
C1 = 3*numPCAcomponents
print(C1)

90


In [9]:
from keras.layers import Input
input_img = Input(shape = (30, 5, 5))

In [10]:
x1=(Conv2D(C1, (1,1), padding='same', activation='relu'))(input_img)
x1 = (Conv2D(C1, (3,3), padding='same', activation='relu'))(x1)
x2 = Conv2D(C1, (1,1), padding='same', activation='relu')(input_img)
x2 = Conv2D(C1, (5,5), padding='same', activation='relu')(x2)
x3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_img)
x3 = Conv2D(C1, (1,1), padding='same', activation='relu')(x3)
output = keras.layers.concatenate([x1, x2, x3], axis = 3)
output = Flatten()(output)
out    = Dense(16, activation='softmax')(output)









Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [12]:
from keras.models import Model
model = Model(inputs = input_img, outputs = out)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30, 5, 5)     0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 90, 5, 5)     2790        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 90, 5, 5)     2790        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 30, 5, 5)     0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (

In [14]:
from keras.optimizers import SGD
epochs = 15
lrate = 0.1
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs, batch_size=32)

Epoch 1/15
29685/29685 [==============================] - 53s 2ms/step - loss: 0.1419 - acc: 0.9593
Epoch 2/15
29685/29685 [==============================] - 53s 2ms/step - loss: 0.0043 - acc: 0.9993
Epoch 3/15
29685/29685 [==============================] - 53s 2ms/step - loss: 0.0018 - acc: 1.0000
Epoch 4/15
29685/29685 [==============================] - 52s 2ms/step - loss: 0.0013 - acc: 1.0000
Epoch 5/15
29685/29685 [==============================] - 53s 2ms/step - loss: 0.0011 - acc: 1.0000
Epoch 6/15
29685/29685 [==============================] - 53s 2ms/step - loss: 0.0010 - acc: 1.0000
Epoch 7/15
29685/29685 [==============================] - 52s 2ms/step - loss: 9.5215e-04 - acc: 1.0000
Epoch 8/15
29685/29685 [==============================] - 53s 2ms/step - loss: 8.9693e-04 - acc: 1.0000
Epoch 9/15
29685/29685 [==============================] - 53s 2ms/step - loss: 8.5207e-04 - acc: 1.0000
Epoch 10/15
29685/29685 [==============================] - 53s 2ms/step - loss: 8.1664e-

In [15]:
import h5py
from keras.models import load_model

In [16]:
model.save('my_model' + str(windowSize) + 'PCA' + str(numPCAcomponents) + "testRatio" + str(testRatio) + '.h5')